In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wG8FEhNmkytG2d_2ujv4SSuc5ae82NZ_tnGXHF_FftPE8eLkjDTatc
Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.metrics.pairwise import cosine_similarity

In [86]:
movies_df = pd.read_csv('/content/gdrive/My Drive/movies.csv')
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)
ratings_df = pd.read_csv('/content/gdrive/My Drive/ratings.csv')
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
print(movies_df.shape)
print(ratings_df.shape)

(9742, 3)
(100836, 4)


In [7]:
print("Total number of ratings :", ratings_df.shape[0])
print("Total number of users   :", len(np.unique(ratings_df.UserID)))
print("Total number of movies  :", len(np.unique(ratings_df.MovieID)))

Total number of ratings : 100836
Total number of users   : 610
Total number of movies  : 9724


In [1]:
movies_df.head()

NameError: name 'movies_df' is not defined

In [ ]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Colaborative Filtering Based RS using Matrix Factorisation

In [8]:
#ratings matrix
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#number of unique users are 610
print(R_df.shape)

(610, 9724)


In [10]:
#normalizing each row by each user's mean
R = R_df.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

Singular Value Decomposition :
$$\begin{equation}R = U\Sigma V^{T}\end{equation}$$


In [21]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 100)

In [12]:
#sigma returned is just the values instead of diagonal matrix
sigma = np.diag(sigma)

In [13]:
#we need to matrix multiply U, sigma, Vt and add the user means back to get the actual ratings prediction
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [14]:
#prediction marix
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,3.471018,0.298326,3.131096,0.229074,0.089782,3.805926,0.164404,0.114733,0.199571,-0.303119,0.157900,0.153204,0.333965,-0.077176,0.208431,1.810503,-0.399183,-0.576370,-0.809329,0.316148,0.273348,-0.384477,0.073127,0.990763,1.172195,0.103039,0.019998,-0.098341,-0.764599,-0.207649,0.379943,-0.993939,0.732793,-0.039052,0.077193,-1.489229,0.018727,0.004495,0.489837,0.155388,...,0.085741,0.049625,-0.087068,-0.107426,-0.087068,-0.099366,-0.087333,-0.271519,-0.034358,-0.039450,-0.034352,-0.262591,-0.019152,0.142914,-0.119932,-0.027916,-0.025002,-0.032307,0.170108,-0.010387,0.006576,-0.013963,-0.036356,-0.010391,-0.046918,-0.003086,-0.010391,-0.003086,-0.003086,0.146507,0.116511,0.101513,0.131509,0.131509,0.116511,0.131509,0.116511,0.116511,0.116511,-0.190738
1,0.354172,0.049318,-0.021538,-0.005594,0.106594,-0.022485,0.030400,0.046609,0.046501,-0.023411,0.152301,0.037980,0.007657,-0.027013,0.079615,0.127508,-0.020197,0.051816,0.192073,0.007075,-0.055754,0.082151,0.047135,0.053183,-0.125774,0.065904,-0.010455,-0.052806,-0.206328,0.012429,-0.003281,-0.190485,-0.127898,0.124983,0.005670,-0.140536,0.007221,0.033223,0.008598,0.018362,...,0.091427,-0.024217,0.038687,0.023762,0.038687,0.030619,0.053069,0.072615,0.033961,0.017718,0.016034,-0.020958,-0.002292,0.032228,0.051290,-0.003198,0.001784,0.001965,0.030419,-0.001386,-0.002254,0.009298,0.032723,0.001422,0.002327,0.001241,0.001422,0.001241,0.001241,0.042222,0.033035,0.028441,0.037629,0.037629,0.033035,0.037629,0.033035,0.033035,0.033035,0.050858
2,0.086552,-0.067687,0.073002,0.010519,0.112911,0.203215,0.081218,0.023242,0.027572,-0.013055,-0.011354,0.027764,0.019756,-0.002696,0.013456,0.009867,-0.008888,0.008947,-0.103591,-0.020461,-0.086577,0.005399,-0.015077,0.008163,-0.079916,0.011937,0.004068,-0.039978,0.019083,0.019289,0.077892,0.066946,0.025366,0.012843,0.004247,0.129275,0.011527,-0.007812,0.026384,0.016357,...,0.020459,-0.007246,0.016077,0.017156,0.016077,0.016314,0.004334,-0.016497,0.006654,-0.010932,-0.008855,0.042973,0.012082,0.000488,0.018159,0.012725,0.011521,0.011859,0.004978,0.011440,0.008245,-0.000550,0.010028,0.010846,0.012534,0.010508,0.010846,0.010508,0.010508,0.008448,0.008755,0.008909,0.008601,0.008601,0.008755,0.008601,0.008755,0.008755,0.008755,0.015282
3,1.165012,0.029818,0.295829,0.098927,0.515800,1.477334,0.606430,0.124794,-0.348266,-0.651972,0.005180,-0.142087,0.198542,-0.021155,0.259008,-0.469384,0.795472,-0.386572,-0.785354,-0.096362,1.830445,0.166434,0.135361,0.763118,1.155823,0.285843,0.208919,0.698130,1.026948,0.208495,0.080775,0.696589,2.056662,0.536887,-0.156384,0.850084,-0.011847,-0.040773,0.167379,0.154354,...,0.007588,0.060526,0.115759,0.104732,0.115759,0.135595,-0.074990,0.134846,-0.022846,0.020082,0.018091,-0.222329,-0.033432,0.086741,0.154287,-0.043034,-0.021127,-0.025388,0.007742,-0.023830,0.008484,0.010128,-0.200240,-0.012607,-0.033908,-0.008346,-0.012607,-0.008346,-0.008346,-0.048636,-0.037789,-0.032366,-0.043213,-0.043213,-0.037789,-0.043213,-0.037789,-0.037789,-0.037789,0.003518
4,1.990547,0.790265,-0.218568,0.067894,0.159691,0.607638,0.236051,0.056216,-0.121986,0.706197,1.115248,-0.151345,0.147861,0.290571,0.025834,0.355434,0.910241,-0.124465,0.231148,0.053896,1.616557,0.200514,0.031919,-0.030917,0.918446,0.213753,-0.016680,0.189216,-0.068217,0.000411,0.396635,1.929685,2.250997,1.324433,0.019618,1.340623,-0.003407,0.011775,0.001188,0.070746,...,-0.030078,0.013624,0.016539,0.010504,0.016539,0.025669,-0.032567,-0.063885,-0.037809,-0.000940,-0.000581,0.029906,-0.010466,0.004047,0.021169,-0.014213,0.014795,0.017224,-0.008500,-0.006719,-0.003507,0.000854

In [15]:
print(preds_df.shape)

(610, 9724)


In [16]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df):
    
    #get and sort the user's predictions
    user_row_number = userID - 1 
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #get the user's data and merge in the movie information
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )
    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    
    #recommend the 5 highest rated predicted movies that the user hasn't seen yet
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',left_on = 'MovieID',right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).iloc[:5, :-1])

    return user_full, recommendations


In [17]:
userID=300
already_rated, predictions = recommend_movies(preds_df, userID, movies_df, ratings_df)
already_rated=already_rated.drop(['UserID', 'MovieID', 'Rating', 'Timestamp'], axis = 1).reset_index(drop=True)
predictions=predictions.drop(['MovieID'], axis = 1).reset_index(drop=True)

User 300 has already rated 32 movies.


In [18]:
already_rated.head()

,Title,Genres
0,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
1,American Beauty (1999),Drama|Romance
2,Boyhood (2014),Drama
3,Interstellar (2014),Sci-Fi|IMAX
4,Black Swan (2010),Drama|Thriller


In [19]:
predictions

,Title,Genres
0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,One Flew Over the Cuckoo's Nest (1975),Drama
2,Memento (2000),Mystery|Thriller
3,Requiem for a Dream (2000),Drama
4,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


Similarity Based RS using Movie-Movie Similarity

In [25]:
sparse_matrix = sparse.csr_matrix((ratings_df.Rating.values, (ratings_df.UserID.values,ratings_df.MovieID.values)))
print('Shape of sparse matrix (user, movie) : ', sparse_matrix.shape)

Shape of sparse matrix (user, movie) :  (611, 193610)


In [26]:
us,mv = sparse_matrix.shape
elem = sparse_matrix.count_nonzero() #this will be equal to number of ratings
print("Sparsity of matrix : {} % ".format(  (1-(elem/(us*mv))) * 100) )

Sparsity of matrix : 99.91475937715747 % 


In [72]:
m_m_sim_sparse = cosine_similarity(X=sparse_matrix.T, dense_output=False)
print("Shape of movie-movie similarity sparse matrix : ",m_m_sim_sparse.shape)

Shape of movie-movie similarity sparse matrix :  (193610, 193610)


In [107]:
sparse.save_npz("m_m_sim_sparse.npz", m_m_sim_sparse)
#m_m_sim_sparse = sparse.load_npz("m_m_sim_sparse.npz")

In [105]:
movie_id = 364
movie_df = movies_df.set_index('MovieID')
print("Movie :",movie_df.loc[movie_id].values[0])
print("Total ratings from users : ",sparse_matrix[:,movie_id].getnnz()) #get the count of explicitly-stored values (nonzeros)
avg_rating = sum(sparse_matrix[:,movie_id].toarray())/sparse_matrix[:,movie_id].getnnz()
print("Average rating out of 5  : ",round(avg_rating[0],1))

Movie : Lion King, The (1994)
Total ratings from users :  172
Average rating out of 5  :  3.9


In [106]:
similarities = m_m_sim_sparse[movie_id].toarray().ravel()
sim_indices = similarities.argsort()[::-1][1:]
similar_df = movies_df.set_index('MovieID')
top10_df = similar_df.loc[sim_indices[:10]]
print(sim_indices[:10])
top10_df

[588 595 500 367 480   2 356  34 586 594]


,Title,Genres
MovieID,,
588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
500,Mrs. Doubtfire (1993),Comedy|Drama
367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
2,Jumanji (1995),Adventure|Children|Fantasy
356,Forrest Gump (1994),Comedy|Drama|Romance|War
34,Babe (1995),Children|Drama
586,Home Alone (1990),Children|Comedy
